In [1]:
using Pkg; Pkg.update()
Pkg.add(url="https://github.com/JuDO-dev/AirBorne.jl#dev")

    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/JuDO-dev/AirBorne.jl#dev`
    Updating `~/Documents/uni-4/FYP/fyp_repo/Project.toml`
  [4a232ea9] ~ AirBorne v1.0.1-DEV `https://github.com/JuDO-dev/AirBorne.jl#dev#dev` ⇒ v1.0.1-DEV `https://github.com/JuDO-dev/AirBorne.jl#dev#dev`
    Updating `~/Documents/uni-4/FYP/fyp_repo/Manifest.toml`
  [4a232ea9] ~ AirBorne v1.0.1-DEV `https://github.com/JuDO-dev/AirBorne.jl#dev#dev` ⇒ v1.0.1-DEV `https://github.com/JuDO-dev/AirBorne.jl#dev#dev`
  [7034ab61] ↑ FastBroadcast v0.3.2 ⇒ v0.3.3
Precompiling project...
  ✓ AirBorne
  1 dependency successfully precompiled in 34 seconds. 343 already precompiled.
    Updating git-repo `https://github.com/JuDO-dev/AirBorne.jl#dev`
   Resolving package versions...
  No Changes to `~/Documents/uni-4/FYP/fyp_repo/Project.toml`
  No Changes to `~/Documents/uni-4/FYP/fyp_repo/Manifest.toml`


In [4]:
using AirBorne.Engines.DEDS: run
using AirBorne
using AirBorne.ETL.YFinance: get_interday_data, get_chart_data, parse_intraday_raw_data
using AirBorne.Markets.StaticMarket: execute_orders!, expose_data, Order, place_order!, executeOrder_CA!
using AirBorne.Strategies.FALM: falm_initialize!, falm_trading_logic!
using AirBorne.Forecast
using Dates
using AirBorne.Structures: summarizePerformance,TimeEvent, ContextTypeA
using AirBorne.ETL.AssetValuation: stockValuation
using DotMaps

using DirectSearch
using Suppressor
using Statistics

function get_annual_returns(data)
    returns = diff(data) ./ data[1:end-1]
    return (1 + mean(returns))^252 - 1
end

unix(x) = string(round(Int, datetime2unix(DateTime(x))))
tickers = ["AAPL", "MSFT", "ISRG", "BABA", "SPY", "EFA", "TLT", "GLD"]
stocks = get_interday_data(tickers, unix("2021-01-01"), unix("2024-01-01"))

feeStructure=Vector{Dict}([Dict("FeeName" => "SaleCommission", "fixedPrice" => 0.0, "variableRate" => 0.02)])
singleExecutionFun(context, order, data) = executeOrder_CA!(context, order, data;defaultFeeStructures=feeStructure,partialExecutionAllowed=false)
my_execute_orders!(context, data) = execute_orders!(context, data; propagateBalanceToPortfolio=true, executeOrder=singleExecutionFun)


function obj(x)
    evaluationEvents = [TimeEvent(t, "data_transfer") for t in sort(unique(stocks.date); rev=true)]

    linear = LinearForecaster(round(Int, x[5]); reparameterise_window = round(Int, x[6]))
    arima = ArimaForecaster(round(Int, x[7]), round(Int, x[8]), 1; reparameterise_window = round(Int, x[9]))
    forecaster = CombinedForecaster([linear, arima], [x[10], x[11]])
    max_lookback = round(Int, max(x[6], x[9]))

    holding_type = x[12] == 1.0 ? :average : :minimum

    falm_init!(context) = falm_initialize!(
        context;
        initialCapital = 10^4,
        lookahead = round(Int, x[2]),
        lpm_order = x[1],
        max_lookback = max_lookback,
        tickers = tickers,
        assetIDs = unique(stocks.assetID),
        transactionCost = 0.02,
        httype = holding_type,
        min_alloc_threshold = x[3],
        min_returns_threshold= x[4],
        forecaster = forecaster
    ) 

    my_context = run(
        stocks,
        falm_init!,
        falm_trading_logic!,
        my_execute_orders!,
        expose_data;
        audit=true,
        verbose=false,
        initialEvents=evaluationEvents,
    )



    dollar_symbol = "FEX/USD"

    usdData = deepcopy(stocks[stocks.symbol .== my_context.extra.tickers[1], :])
    usdData[!, "assetID"] .= dollar_symbol
    usdData[!, "exchangeName"] .= "FEX"
    usdData[!, "symbol"] .= "USD"
    usdData[!, [:close, :high, :low, :open]] .= 1.0
    usdData[!, [:volume]] .= 0
    OHLCV_data = vcat(stocks, usdData)

    results = @suppress summarizePerformance(OHLCV_data, my_context; includeAccounts=false, riskFreeRate=0.04)
    return -get_annual_returns(results.dollarValue)
end

initial_points = [[1.0, 1.0, 0.70, 0.00015, 3.0, 30.0, 2.0, 1.0, 45.0, 0.50, 0.50, 1.0],
                  [2.0, 2.0, 0.50, 0.00025, 1.0, 40.0, 4.0, 3.0, 15.0, 0.80, 0.20, 2.0],
                  [3.0, 2.0, 0.99, 0.00045, 6.0, 10.0, 1.0, 2.0, 60.0, 0.20, 0.80, 1.0],
                  [4.0, 3.0, 0.00, 0.00000, 1.0, 50.0, 1.0, 3.0, 50.0, 0.50, 0.50, 2.0]]
granularity = [1.0, 1.0, 0.01, 0.00001, 1.0, 1.0, 1.0, 1.0, 1.0, 0.01, 0.01, 1.0]

# Define the search space
geq(x) = all(x .>= 0)
lookahead(x) = x[2] >= 1.0
min_alloc_threshold(x) = x[3] <= 1.00
linear_lookback(x) = x[5] >= 1.0
linear_reparameterise_window(x) = x[6] >= (x[2] + x[5]) 
ar_order(x) = x[7] >= 0.9
arima_reparameterise_window(x) = x[9] >= (x[7] + x[8] + x[2])
httype(x) = (x[12] == 1.0) || (x[12] == 2.0)

for point in initial_points
    println("Optimizing for initial point: ", point)
    if !geq(point) || !lookahead(point) || !min_alloc_threshold(point) || !linear_lookback(point) || !linear_reparameterise_window(point) || !ar_order(point) || !arima_reparameterise_window(point) || !httype(point)
        println(point, "is an invalid initial point")
        continue
    end

    falm_problem = DSProblem(12, objective=obj, granularity=granularity)
    SetInitialPoint(falm_problem, point)
    AddExtremeConstraint(falm_problem, geq)
    AddExtremeConstraint(falm_problem, lookahead)
    AddExtremeConstraint(falm_problem, min_alloc_threshold)
    AddExtremeConstraint(falm_problem, linear_lookback)
    AddExtremeConstraint(falm_problem, linear_reparameterise_window)
    AddExtremeConstraint(falm_problem, ar_order)
    AddExtremeConstraint(falm_problem, arima_reparameterise_window)
    try
        Optimize!(falm_problem)
    catch e
        @error "Optimization failed for initial point: $(point)" exception=(e, catch_backtrace())
        continue
    end
end




Optimizing for initial point: [1.0, 1.0, 0.7, 0.00015, 3.0, 30.0, 2.0, 1.0, 45.0, 0.5, 0.5, 1.0]
MADS Run Summary
------------------------------------------------------------------------------------------------
Feasible Solution           [0.0, 2.0, 0.69, 0.0002, 3.0, 27.0, 1.0, 1.0, 41.0, 0.51, 0.49, 1.0]
Feasible Cost               -0.06585861516984015
Infeasible Solution         nothing
Infeasible Cost             nothing

Iterations                  68
Function Evaluations        466
Cache hits                  965
Optimization Status         Mesh Precision limit

Runtime                     7755.94894194603
Search Time                 3.483399999999999e-5
Poll Time                   0.10657097099999996
Blackbox Evaluation Time    7731.030990555984
Optimizing for initial point: [2.0, 2.0, 0.5, 0.00025, 1.0, 40.0, 4.0, 3.0, 15.0, 0.8, 0.2, 2.0]
Optimizing for initial point: [3.0, 2.0, 0.99, 0.00045, 6.0, 10.0, 1.0, 2.0, 60.0, 0.2, 0.8, 1.0]


┌ Error: Optimization failed for initial point: [2.0, 2.0, 0.5, 0.00025, 1.0, 40.0, 4.0, 3.0, 15.0, 0.8, 0.2, 2.0]
│   exception = (DomainError(-0.33223542622559693, "log will only return a complex result if called with a complex argument. Try log(Complex(x))."), Union{Ptr{Nothing}, Base.InterpreterIP}[Ptr{Nothing} @0x0000000121c8504c, Ptr{Nothing} @0x0000000122375090, Ptr{Nothing} @0x0000000175f89c3f, Ptr{Nothing} @0x0000000175f89d32, Ptr{Nothing} @0x000000010c009003, Ptr{Nothing} @0x0000000175f57f52, Ptr{Nothing} @0x0000000175f5aa9e, Ptr{Nothing} @0x00000001762f983c, Ptr{Nothing} @0x00000001762f9993, Ptr{Nothing} @0x000000010c009003, Ptr{Nothing} @0x0000000175f51ef3, Ptr{Nothing} @0x0000000175f51f2a, Ptr{Nothing} @0x000000010c009003, Ptr{Nothing} @0x00000001762f545f, Ptr{Nothing} @0x00000001762f7db4, Ptr{Nothing} @0x00000001762f7e6d, Ptr{Nothing} @0x000000010c009003, Ptr{Nothing} @0x000000016b96ce52, Ptr{Nothing} @0x000000016b96dbdb, Ptr{Nothing} @0x000000016b978b41, Ptr{Nothing} @0x